In [ ]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
# split the dataframe into 70-30 train-test sets
train_size = int(0.7 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]
# save to csv the train and test sets
train_df.to_csv('energy_data_train.csv', index=False)
test_df.to_csv('energy_data_test.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv('energy_data_.csv')
column_to_move = 'Energy'
df = df[[col for col in df.columns if col != column_to_move] + [column_to_move]]
df.to_csv('energy_data_.csv', index=False)